In [2]:
import json
import altair as alt
from altair import expr, datum
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import requests
import re

In [3]:
import colorsys
from matplotlib.colors import to_hex, to_rgb


def scale_lightness(rgb, scale_l):
    rgbhex = False
    if "#" in rgb:
        rgb = to_rgb(rgb)
        rgbhex = True
    # convert rgb to hls
    h, l, s = colorsys.rgb_to_hls(*rgb)
    # manipulate h, l, s values and return as rgb
    c = colorsys.hls_to_rgb(h, min(1, l * scale_l), s=s)
    if rgbhex:
        c = to_hex(c)
    return c

In [4]:
LOCAL = True

if LOCAL:
    local_suffix = "_local"
else:
    local_suffix = ""

In [5]:
%%capture pwd
!pwd

In [6]:
uid = pwd.stdout.split("/")[-1].split("\r")[0]
eco_git_home = (
    "https://raw.githubusercontent.com/EconomicsObservatory/ECOvisualisations/main/"
)
eco_git_path = eco_git_home + "articles/" + uid + "/data/"
vega_embed = requests.get(eco_git_home + "guidelines/html/vega-embed.html").text
colors = json.loads(
    requests.get(eco_git_home + "guidelines/colors/eco-colors.json").content
)
category_color = json.loads(
    requests.get(eco_git_home + "guidelines/colors/eco-category-color.json").content
)
hue_color = json.loads(
    requests.get(eco_git_home + "guidelines/colors/eco-single-hue-color.json").content
)
mhue_color = json.loads(
    requests.get(eco_git_home + "guidelines/colors/eco-multi-hue-color.json").content
)
div_color = json.loads(
    requests.get(eco_git_home + "guidelines/colors/eco-diverging-color.json").content
)
config = json.loads(
    requests.get(eco_git_home + "guidelines/charts/eco-global-config.json").content
)
height = config["height"]
width = config["width"]
uid, height, width

('how-does-climate-change-shape-poverty-inequality-and-economic-opportunity',
 300,
 500)

# Fig 1

In [7]:
# https://www.eia.gov/outlooks/aeo/data/browser/#/?id=1-IEO2019&region=0-0&cases=Reference&start=2010&end=2050&f=A&linechart=~Reference-d080819.11-1-IEO2019~Reference-d080819.25-1-IEO2019&ctype=linechart&sourcekey=0
df = pd.read_csv("raw/Energy_use_(Case_Reference).csv", skiprows=4)
df.columns = ["year", "OECD", "non-OECD"]
df = df.set_index("year").stack().reset_index()
df.columns = ["year", "oecd", "value"]

In [8]:
f = "fig1_energy-consumption"
f1 = eco_git_path + f + ".csv"
df.to_csv("data/" + f + ".csv")
f += local_suffix
open("visualisation/" + f + ".html", "w").write(
    vega_embed.replace(
        "JSON_PATH", f1.replace("/data/", "/visualisation/").replace(".csv", ".json")
    )
)
if LOCAL:
    f1 = df
readme = "### " + f + '\n!["' + f + '"](visualisation/' + f + '.png "' + f + '")\n\n'
df.head()

,year,oecd,value
0,2050,OECD,286.8160
1,2050,non-OECD,623.8824
2,2049,OECD,284.6088
3,2049,non-OECD,614.4289
4,2048,OECD,282.6116


In [9]:
base = alt.Chart(f1).encode(
    x=alt.X(
        "year:Q",
        axis=alt.Axis(
            grid=False,
            titleAlign="center",
            titleAnchor="middle",
            title="",
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
            tickCount=5,
            orient="bottom",
            labelAngle=0,
            format=".0f",
            # zindex=1,
            # offset=-43
        ),
    )
)
area = base.mark_area(
    interpolate="monotone",
    fillOpacity=0.7,
    line={"color": colors["eco-dark-blue"]},
    color=alt.Gradient(
        gradient="linear",
        stops=[
            alt.GradientStop(color="white", offset=0),
            alt.GradientStop(color=colors["eco-dark-blue"], offset=0.7),
        ],
        x1=0.8,
        x2=0,
        y1=0.8,
        y2=0,
    ),
).encode(
    y=alt.Y(
        "value:Q",
        stack=True,
        axis=alt.Axis(
            grid=True,
            title="quadrillion Bristish thermal units",
            titleAnchor="start",
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
            titleX=-3,
            titleY=-3,
            titleBaseline="bottom",
            titleAngle=0,
            titleAlign="left",
            tickCount=4,
            format=".0f",
        ),
    ),
    color=alt.Color(
        "oecd:N",
        legend=alt.Legend(title=None,orient='top-left',labelColor=colors["eco-gray"]),
        scale=alt.Scale(range=[colors["eco-turquiose"], colors["eco-mid-blue"]]),
    ),
)
title = alt.TitleParams(
    "Coal consumption, 1750-2000",
    subtitle=["Source: Stephan Heblich, based on Warde (2007)"],
    anchor="start",
    align="left",
    dx=5,
    dy=-5,
    fontSize=12,
    subtitleFontSize=11,
    subtitleFontStyle="italic",
)
layer1 = (
    ((area).properties(height=300, width=400))
    .configure_view(stroke=None)
    .properties(title=title)
)
layer1.save("visualisation/" + f + ".json")
layer1.save("visualisation/" + f + ".png")
open("README.md", "w").write(readme)
layer1

Fontconfig error: Cannot load default config file


alt.Chart(...)

In [10]:
base = alt.Chart(f1).encode(
    x=alt.X(
        "year:Q",
        axis=alt.Axis(
            grid=False,
            titleAlign="center",
            titleAnchor="middle",
            title="",
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
            tickCount=5,
            titleFontSize=12,
            labelFontSize=12,
            orient="bottom",
            labelAngle=0,
            format=".0f",
            # zindex=1,
            # offset=-43
        ),
    )
)
area = base.mark_area(
    interpolate="monotone",
    fillOpacity=0.7,
    line={"color": colors["eco-dark-blue"]},
    color=alt.Gradient(
        gradient="linear",
        stops=[
            alt.GradientStop(color="white", offset=0),
            alt.GradientStop(color=colors["eco-dark-blue"], offset=0.7),
        ],
        x1=0.8,
        x2=0,
        y1=0.8,
        y2=0,
    ),
).encode(
    y=alt.Y(
        "value:Q",
        stack=True,
        axis=alt.Axis(
            grid=False,
            title="quadrillion Bristish thermal units",
            titleAnchor="start",
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
            titleX=-3,
            titleY=-3,
            titleFontSize=12,
            labelFontSize=12,
            titleBaseline="bottom",
            titleAngle=0,
            titleAlign="left",
            tickCount=4,
            format=".0f",
        ),
    ),
    color=alt.Color(
        "oecd:N",
        legend=alt.Legend(title=None,orient='top-left',labelColor=colors["eco-gray"]),
        scale=alt.Scale(range=[colors["eco-turquiose"], colors["eco-mid-blue"]]),
    ),
)
title = alt.TitleParams(
    "Coal consumption, 1750-2000",
    subtitle=["Source: Stephan Heblich, based on Warde (2007)"],
    anchor="start",
    align="left",
    dx=5,
    dy=-5,
    fontSize=14,
    subtitleFontSize=12,
    color=colors["eco-dot"],
    subtitleColor=colors["eco-dot"],
    subtitleFontStyle="italic",
)
layer1 = (
    ((area).properties(height=300, width=400))
    .configure(font="Georgia", background=colors["eco-background"])
    .configure_view(stroke=None)
    .properties(title=title)
)
layer1.save("visualisation/" + f + "_dark.json")
layer1.save("visualisation/" + f + "_dark.png")
readme=re.sub(f, f+'_dark', readme)
open("README.md", "a").write(readme)
layer1

Fontconfig error: Cannot load default config file


alt.Chart(...)